## 3. Search Data

In [4]:
import os
import json
import requests

from rasterio.features import bounds as featureBounds

from ipyleaflet import Map, basemaps, TileLayer, basemap_to_tiles, GeoJSON

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
# Endpoint variables
titiler_endpoint = "{ENDPOINT}"
stac_endpoint = "https://earth-search.aws.element84.com/v0/search"

# Make sure both are up
assert requests.get(f"{titiler_endpoint}/docs").status_code == 200
assert requests.get(stac_endpoint).status_code == 200

More info: https://github.com/radiantearth/stac-api-spec for more documentation about the stac API

1. AOI

You can use geojson.io to define your search AOI

In [6]:
geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -2.83447265625,
              4.12728532324537
            ],
            [
              2.120361328125,
              4.12728532324537
            ],
            [
              2.120361328125,
              8.254982704877875
            ],
            [
              -2.83447265625,
              8.254982704877875
            ],
            [
              -2.83447265625,
              4.12728532324537
            ]
          ]
        ]
      }
    }
  ]
}

bounds = featureBounds(geojson)

m = Map(
    basemap=basemaps.OpenStreetMap.Mapnik,
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=6
)

geo_json = GeoJSON(data=geojson)
m.add_layer(geo_json)
m

Map(center=[6.191134014061623, -0.3570556640625], controls=(ZoomControl(options=[&#39;position&#39;, &#39;zoom_in_text&#39;, &#39;…

2. Define dates and other filters

In [7]:
start = datetime.datetime.strptime("2019-01-01", "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")
end = datetime.datetime.strptime("2019-12-11", "%Y-%m-%d").strftime("%Y-%m-%dT23:59:59Z")

# POST body
query = {
    "collections": ["sentinel-s2-l2a-cogs"],
    "datetime": f"{start}/{end}",
    "query": {
        "eo:cloud_cover": {
            "lt": 3
        },
        "sentinel:data_coverage": {
            "gt": 10
        }
    },
    "intersects": geojson["features"][0]["geometry"],
    "limit": 1000,
    "fields": {
      'include': ['id', 'properties.datetime', 'properties.eo:cloud_cover'],  # This will limit the size of returned body
      'exclude': ['assets', 'links']  # This will limit the size of returned body
    },
    "sortby": [
        {
            "field": "properties.eo:cloud_cover",
            "direction": "desc"
        },
    ]
}

# POST Headers
headers = {
    "Content-Type": "application/json",
    "Accept-Encoding": "gzip",
    "Accept": "application/geo+json",
}

data = requests.post(stac_endpoint, headers=headers, json=query).json()
print("Results context:")
print(data["context"])

sceneid = [f["id"] for f in data["features"]]
cloudcover = [f["properties"]["eo:cloud_cover"] for f in data["features"]]
dates = [f["properties"]["datetime"][0:10] for f in data["features"]]

Results context:
{&#39;page&#39;: 1, &#39;limit&#39;: 1000, &#39;matched&#39;: 408, &#39;returned&#39;: 408}


In [92]:
data["features"][0]

{'bbox': [0.2927641695171779,
  4.429965030479934,
  1.2856914562725514,
  5.4257779507944806],
 'geometry': {'coordinates': [[[0.2967780962947474, 4.429965030479934],
    [0.2927641695171779, 5.422153918603194],
    [1.28314366979741, 5.4257779507944806],
    [1.2856914562725514, 4.43292307525344],
    [0.2967780962947474, 4.429965030479934]]],
  'type': 'Polygon'},
 'id': 'S2B_31NBF_20190109_0_L2A',
 'collection': 'sentinel-s2-l2a-cogs',
 'type': 'Feature',
 'properties': {'datetime': '2019-01-09T10:29:38Z', 'eo:cloud_cover': 2.98}}

In [8]:
m = Map(
    basemap=basemaps.OpenStreetMap.Mapnik,
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=8
)

geo_json = GeoJSON(
    data=data,
    style={
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
m.add_layer(geo_json)
m

Map(center=[6.191134014061623, -0.3570556640625], controls=(ZoomControl(options=[&#39;position&#39;, &#39;zoom_in_text&#39;, &#39;…

In [94]:
# Print what band are available
from rio_tiler_pds.sentinel.aws.sentinel2 import default_l2a_bands
print(default_l2a_bands)

('B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11', 'B12', 'B8A')


In [11]:
# Fetch TileJSON
# For this example we use the first `sceneid` return from the STAC API
# and we sent the Bands to B04,B03,B02 which are red,green,blue
print(sceneid[4])
data = requests.get(f"{titiler_endpoint}/scenes/sentinel/tilejson.json?sceneid={sceneid[4]}&bands=B04,B03,B02&rescale=0,2000").json()
print(data)

S2B_31NCG_20190926_0_L2A
{&#39;tilejson&#39;: &#39;2.2.0&#39;, &#39;name&#39;: &#39;S2B_31NCG_20190926_0_L2A&#39;, &#39;version&#39;: &#39;1.0.0&#39;, &#39;scheme&#39;: &#39;xyz&#39;, &#39;tiles&#39;: [&#39;https://c50qa6bhpe.execute-api.us-west-2.amazonaws.com/sentinel/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?sceneid=S2B_31NCG_20190926_0_L2A&amp;bands=B04%2CB03%2CB02&amp;rescale=0%2C2000&#39;], &#39;minzoom&#39;: 8, &#39;maxzoom&#39;: 14, &#39;bounds&#39;: [1.1920033127639242, 5.337108286531346, 2.1859046942193197, 6.332392577649882], &#39;center&#39;: [1.688954003491622, 5.834750432090614, 8]}


In [12]:
bounds = data["bounds"]
m = Map(
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=10
)

tiles = TileLayer(
    url=data["tiles"][0],
    min_zoom=data["minzoom"],
    max_zoom=data["maxzoom"],
    opacity=1
)
m.add_layer(tiles)
m

Map(center=[5.834750432090614, 1.688954003491622], controls=(ZoomControl(options=[&#39;position&#39;, &#39;zoom_in_text&#39;, …